# Assignment 1: Web Scraping

## Objective

Data scientists often need to crawl data from websites and turn the crawled data (HTML pages) to structured data (tables). Thus, web scraping is an essential skill that every data scientist should master. In this assignment, you will learn the followings:


* How to use [requests](http://www.python-requests.org/en/master/) to download HTML pages from a website?
* How to select content on a webpage with [lxml](http://lxml.de/)? 

You can either use Spark DataFrame or [pandas.DataFrame](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.html) to do the assignment. In comparison, pandas.DataFrame has richer APIs, but is not good at distributed computing.

## Preliminary

If this is your first time to write a web scraper, you need to learn some basic knowledge of HTML, DOM, and XPath. I found that this is a good resource: [https://data-lessons.github.io](https://data-lessons.github.io/library-webscraping-DEPRECATED/). Please take a look at

* [Selecting content on a web page with XPath
](https://data-lessons.github.io/library-webscraping-DEPRECATED/xpath/)
* [Web scraping using Python: requests and lxml](https://data-lessons.github.io/library-webscraping-DEPRECATED/04-lxml/). 

Please let me know if you find a better resource. I'll share it with the other students.

## Overview

Imagine you are a data scientist working at SFU. One day, you want to analyze CS faculty data and answer two interesting questions:

1. Who are the CS faculty members?
2. What are their research interests?

To do so, the first thing is to figure out what data to collect.

## Task 1: SFU CS Faculty Members

You find that there is a web page in the CS school website, which lists all the faculty members as well as their basic information. 

In Task 1, your job is to write a web scraper to extract the faculty information from this page: [https://www.sfu.ca/computing/people/faculty.html](https://www.sfu.ca/computing/people/faculty.html).




### (a) Crawling Web Page

A web page is essentially a file stored in a remote machine (called web server). You can use [requests](http://www.python-requests.org/en/master/) to open such a file and read data from it. Please complete the following code to download the HTML page and save it as a text file (like [this](./faculty.txt)). 

In [2]:
import requests

# 1. Download the webpage
page = requests.get("https://www.sfu.ca/computing/people/faculty.html")

# 2. Save it as a text file (named faculty.txt)
file = open("faculty.txt","wb")
file.write(page.text.encode('utf8'))
file.close()

### (b) Extracting Structured Data

An HTML page follows the Document Object Model (DOM). It models an HTML page as a tree structure wherein each node is an object representing a part of the page. The nodes can be searched and extracted programmatically using XPath. Please complete the following code to transform the above HTML page to a CSV file (like [this](./faculty_table.csv)). 

In [3]:
import lxml.html 
import pandas as pd

# 1. Open faculty.txt
file = open("faculty.txt","r")
treestruc = lxml.html.fromstring(file.read())
# 2. Parse the HTML page as a tree structure
allprofessors = treestruc.xpath('//*[contains(@class,"textimage section")]')
name = []
rank = []
area = []
profile = [] 
homepage = []
# 3. Extract related content from the tree using XPath
for prof in allprofessors:
    #name and rank
    namerank_xpath = prof.xpath('.//div[@class="text"]/h4/text()')
    namerank_sep = namerank_xpath[0].split(", ")
    namerank_sep[1] = namerank_sep[1].strip("\xa0")
    name.append(namerank_sep[0])
    rank.append(namerank_sep[1])
    #area
    if namerank_sep[0] == "Fred Popowich":
        area_name = namerank_xpath[1]
        area.append(area_name)
    else:     
        area_xpath = prof.xpath('.//p[1]/text()')
        area_name = area_xpath[0].strip(": ").strip("\xa0")
        area.append(area_name)
    #profile and homepage
    profile_xpath = prof.xpath('.//p[2]/a[1]/@href')
    if len(prof.xpath('.//p[2]/a[1]/@href')) == 0:
        if len(prof.xpath('.//p[3]/a[1]/@href')) != 0:
            profile_xpath = prof.xpath('.//p[3]/a[1]/@href')
        elif len(prof.xpath('.//p[1]/a[1]/@href')) != 0:
            profile_xpath = prof.xpath('.//p[1]/a[1]/@href')
            
    if profile_xpath[0].startswith('http://'):
        profile_link = profile_xpath[0]
    else:
        profile_link = 'http://www.sfu.ca' + profile_xpath[0]
    profile.append(profile_link)
    
    #homepage
    homepage_xpath = prof.xpath('.//p[2]/a[2]/@href')
    if len(prof.xpath('.//p[2]/a[2]/@href')) == 0:
        if len(prof.xpath('.//p[3]/a[2]/@href')) != 0:
            homepage_xpath = prof.xpath('.//p[3]/a[2]/@href')
        elif len(prof.xpath('.//p[1]/a[2]/@href')) != 0:
            homepage_xpath = prof.xpath('.//p[1]/a[2]/@href')
        else:
            homepage_xpath = ' '
    homepage.append(homepage_xpath[0])

dict = {"name":name,"rank":rank,"area":area,"profile":profile,"homepage":homepage}
prof_df = pd.DataFrame(dict)
# 4. Save the extracted content as an csv file (named faculty_table.csv)
prof_df.to_csv('faculty_table.csv',index=False)

## Task 2: Research Interests

Suppose you want to know the research interests of each faculty. However, the above crawled web page does not contain such information. 

### (a) Crawling Web Page

You notice that such information can be found on the profile page of each faculty. For example, you can find the research interests of Dr. Jiannan Wang from [http://www.sfu.ca/computing/people/faculty/jiannanwang.html](http://www.sfu.ca/computing/people/faculty/jiannanwang.html). 


Please complete the following code to download the profile pages and save them as text files. There are 60 faculties, so you need to download 60 web pages in total. 

In [4]:
import requests

file = open('faculty_table.csv',"r")
profile = prof_df['profile'].tolist()
name = prof_df['name'].tolist()
# 1. Download the profile pages of 60 faculties
for i in range(len(profile)):
    page = requests.get(profile[i])
# 2. Save each page as a text file
    file = open(name[i].replace(' ','')+".txt","wb")
    file.write(page.text.encode('utf8'))
    file.close()

### (b) Extracting Structured Data

Please complete the following code to extract the research interests of each faculty, and generate a file like [this](./faculty_more_table.csv). 

In [41]:
import lxml.html 
import pandas as pd

li = ["Research Interests","Research interests"]
research_interest = []
path = ['//div[@class="parsys_column cq-colctrl-lt0-c1"]/div[2]/div','//div[@class="parsys_column cq-colctrl-lt0-c0"]/div[4]/div[1]','//div[@class="parsys_column cq-colctrl-lt0-c1"]/div[3]/div','//div[@class="parsys_column cq-colctrl-lt0-c1"]/div[4]/div']
for i in range(len(profile)):
# 1. Open each text file and parse it as a tree structure 
    file = open(name[i].replace(' ','')+".txt","r",encoding="utf8")
    treestruc = lxml.html.fromstring(file.read())

    for j in range(len(path)):
        if len(treestruc.xpath(path[j]+"/h2/text()")) != 0 and treestruc.xpath(path[j]+"/h2/text()")[0] in li:
            temp = treestruc.xpath(path[j]+"/ul//li")
# 2. Extract the research interests from each tree using XPath
            ri_path = [ x.text_content().replace('\xa0','').replace('\n','') for x in temp]
            while "\n" in ri_path: ri_path.remove("\n")
            break
        else:
            ri_path =[]
    research_interest.append(ri_path)
    
re_interest = pd.Series(research_interest)
# 3. Add the extracted content to faculty_table.csv
prof_df['research_interests'] = re_interest.values
# 4. Generate a new CSV file, named faculty_more_table.csv
prof_df.to_csv('faculty_more_table.csv',index=False)           

## Submission

Complete the code in this [notebook](A1.ipynb), and submit it to the CourSys activity `Assignment 1`.